In [36]:
# Se importan las librerías a utilizar en el ejercicio
import pandas as pd
from dask import dataframe as df1

**Dask DataFrame**

Un Dask DataFrame es un gran DataFrame paralelo compuesto por muchos *Pandas DataFrames* más pequeños, divididos a lo largo del índice. Una operación de Dask DataFrame desencadena muchas operaciones en los Pandas DataFrames constituyentes, argumentado lo siguiente en su documentación:

*Dask DataFrame se usa en situaciones en las que se necesitan comúnmente Pandas, generalmente cuando Pandas falla debido al tamaño de los datos o la velocidad de cálculo*

Por el tamañano del archivo (alcanzando los 20 gb), ser más eficiente que trabajar el dataset en chunks directamente con pandas, y ocupar pandas por debajo, se elije esta librería para el ejercicio.

![Dask](https://miro.medium.com/max/412/1*Sjy6G18SAMFEKbJfBYEU8g.png)

In [41]:
# Se lee el archivo, configurando número de bytes por los que cortar el archivo
dask_df = df1.read_csv(
    "opi analytics/all_data.csv",
    blocksize=25e6)


1. ¿Cuántos registros hay?

In [4]:
num_registros = dask_df.producto.size.compute()

print("Hay", num_registros, "registros")

Hay 62530715 registros


2. ¿Cuántas categorías?

In [38]:
dask_df.head()

,producto,presentacion,marca,categoria,catalogo,precio,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio,latitud,longitud
0,CUADERNO FORMA ITALIANA,96 HOJAS PASTA DURA. CUADRICULA CHICA,ESTRELLA,MATERIAL ESCOLAR,UTILES ESCOLARES,25.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
1,CRAYONES,CAJA 12 CERAS. JUMBO. C.B. 201423,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,27.5,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
2,CRAYONES,CAJA 12 CERAS. TAMANO REGULAR C.B. 201034,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,13.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
3,COLORES DE MADERA,CAJA 12 PIEZAS LARGO. TRIANGULAR. C.B. 640646,PINCELIN,MATERIAL ESCOLAR,UTILES ESCOLARES,46.9,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417
4,COLOR LARGO,CAJA 36 PIEZAS. CON SACAPUNTAS. 68-4036,CRAYOLA,MATERIAL ESCOLAR,UTILES ESCOLARES,115,2011-05-18 00:00:00.000,ABASTECEDORA LUMEN,PAPELERIAS,ABASTECEDORA LUMEN SUCURSAL VILLA COAPA,CANNES No. 6 ESQ. CANAL DE MIRAMONTES,DISTRITO FEDERAL,TLALPAN,19.29699,-99.125417


In [6]:
info_df = dask_df.describe(include=['object', 'category']).compute()

In [7]:
info_df

,producto,presentacion,marca,categoria,catalogo,fechaRegistro,cadenaComercial,giro,nombreComercial,direccion,estado,municipio
unique,1107,8244,2079,41,12,7487339,705,21,3895,6526,33,206
count,62530715,62530715,62530715,61643377,62530487,62530715,62529531,62530428,62530715,62530715,62515661,62515661
top,REFRESCO,1 KG. GRANEL,S/M,MEDICAMENTOS,BASICOS,2014-12-01 00:00:00.000,WAL-MART,TIENDA DE AUTOSERVICIO,WAL MART,BOULEVARD JOSE TORRES LANDA NO. 4137 ENTRE BLV...,DISTRITO FEDERAL,LEON
freq,1247981,868870,19814738,11485813,33321024,116484,8643133,53084163,810219,197304,11284102,1908469


In [14]:
print("Existen", info_df['categoria'][0] ,"diferentes caregorías")

Existen 41 diferentes caregorías


3. ¿Cuántas cadenas comerciales están siendo monitoreadas (y, por lo tanto, reportadas
en esa base de datos)?

In [17]:
print(
    "Se han reportado",
    info_df["cadenaComercial"][1],
    "cadenas, siendo",
    info_df["cadenaComercial"][0],
    "cadenas comerciales distintas",
)


Se han reportado 62529531 cadenas, siendo 705 cadenas comerciales distintas


4. ¿Cuáles son los productos más monitoreados en cada estado de la república?

Pasos realizados:
* Se seleccionan solo los campos a analizar
* Con dask se analiza cuantas veces aparece un producto en un estado
* El resultado se guarda en estructura pandas.Series
* Se convierte a Dataframe para mejor manejo
* Se agrupa por estado y se selecciona el máximo

In [45]:
df_dask_subset_proest = dask_df[['producto', 'estado']]

In [46]:
df_productos_estado = (
    df_dask_subset_proest.groupby("estado")["producto"]
    .value_counts()
    .drop_duplicates()
    .compute()
)


In [201]:
df_productos_estado_conv = df_productos_estado.to_frame()
df_productos_estado_conv = df_productos_estado_conv.rename(
    columns={"producto": "count"}
)
df_productos_estado_conv.reset_index(inplace=True)


In [209]:
df_productos_estado_conv.loc[df_productos_estado_conv.groupby(["estado"])["count"].idxmax()]     

,estado,producto,count
253,AGUASCALIENTES,FUD,12005
5932,BAJA CALIFORNIA,REFRESCO,37243
6004,BAJA CALIFORNIA SUR,REFRESCO,27770
6029,CAMPECHE,FUD,12960
6081,CHIAPAS,REFRESCO,14452
6109,CHIHUAHUA,DETERGENTE P/ROPA,15982
710,COAHUILA DE ZARAGOZA,FUD,28876
1074,COLIMA,DETERGENTE P/ROPA,17193
1930,DISTRITO FEDERAL,REFRESCO,287463
5828,DURANGO,DETERGENTE P/ROPA,9696


5. ¿Cuál es la cadena comercial con mayor variedad de productos monitoreados?

Pasos Realizados:

* Se seleccionan solo las columnas a trabajar
* Se hace un agrupamiento y contedo de productos
* El resultado se guarda en estructura pandas.Series
* Se convierte a Dataframe para mejor manejo
* Se hace un conteo de cuantos veces aparecio
* Se selecciona el primer elemento que es el que tiene más apariciones

In [210]:
df_dask_subset_procad = dask_df[['producto', 'cadenaComercial']]

In [211]:
df_productos_cadena = (
    df_dask_subset_procad.groupby("cadenaComercial")["producto"]
    .value_counts()
    .drop_duplicates()
    .compute()
)


In [216]:
df_productos_cadena_conv = df_productos_cadena.to_frame()
df_productos_cadena_conv = df_productos_cadena_conv.rename(
    columns={"producto": "count"}
)
df_productos_cadena_conv.reset_index(inplace=True)

In [249]:
cadenaComercial_mayor = df_productos_cadena_conv.cadenaComercial.value_counts()[[0]]
print(
    "La cadena comercial con más productos es",
    cadenaComercial_mayor.index[0],
    "con",
    cadenaComercial_mayor[0],
    "productos monitoreados",
)


La cadena comercial con más productos es BODEGA AURRERA con 708 productos monitoreados


**Encuentra algún dato curioso en los datos y comunícalo en un slide de powerpoint.**

In [268]:
datos_refresco = df_productos_estado_conv.loc[
    df_productos_estado_conv["producto"] == "REFRESCO"
].sort_values("count", ascending=False)

datos_refresco = datos_refresco.rename(
    columns={"count": "apariciones"}
)

datos_refresco.tail(6)

,estado,producto,apariciones
6044,CAMPECHE,REFRESCO,11333
6200,MORELOS,REFRESCO,9632
434,AGUASCALIENTES,REFRESCO,9481
6177,GUERRERO,REFRESCO,8932
5847,DURANGO,REFRESCO,8579
6218,NAYARIT,REFRESCO,8003
